In [4]:
import os
import errno
import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks

import deepcell
from deepcell import model_zoo
from deepcell import losses
from deepcell.utils.data_utils import get_data
from deepcell.utils.train_utils import rate_scheduler
from deepcell.utils.tracking_utils import load_trks
from deepcell.utils.tracking_utils import save_trks
from deepcell.utils.retinanet_anchor_utils import get_anchor_parameters
from deepcell.callbacks import RedirectModel, Evaluate
from deepcell.image_generators import RetinaMovieDataGenerator, RetinaNetGenerator
# from deepcell.model_zoo import shapemask_box

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Helper Functions
From `shape_mask` branch

In [5]:
def train_model(model,
                model_dir=None,
                model_name=None,
                train_dict=None,
                test_dict=None,
                batch_size=1,
                num_classes=1,
                fpb=1,
                backbone_levels=None,
                pyramid_levels=None,
                anchor_params=None,
                n_epoch=16,
                optimizer=Adam(lr=1e-5, clipnorm=0.001),
                lr_sched = rate_scheduler(lr=1e-5, decay=0.99)
                ):
    
    if fpb == 1:
        datagen = RetinaNetGenerator(
            rotation_range=180,
            zoom_range=(0.8, 1.2),
            horizontal_flip=True,
            vertical_flip=True)

        datagen_val = RetinaNetGenerator()

        train_data = datagen.flow(
            train_dict,
            batch_size=batch_size,
            include_masks=True,
            include_final_detection_layer=True,
            pyramid_levels=pyramid_levels,
            anchor_params=anchor_params)

        val_data = datagen_val.flow(
            test_dict,
            batch_size=batch_size,
            include_masks=True,
            include_final_detection_layer=True,
            pyramid_levels=pyramid_levels,
            anchor_params=anchor_params)
    
    else:
        datagen = RetinaMovieDataGenerator(
            rotation_range=180,
            zoom_range=(0.8, 1.2),
            horizontal_flip=True,
            vertical_flip=True)

        datagen_val = RetinaMovieDataGenerator()

        train_data = datagen.flow(
            train_dict,
            batch_size=batch_size,
            include_masks=True,
            include_final_detection_layer=True,
            frames_per_batch=fpb,
            pyramid_levels=pyramid_levels,
            anchor_params=anchor_params)

        val_data = datagen_val.flow(
            test_dict,
            batch_size=batch_size,
            include_masks=True,
            include_final_detection_layer=True,
            frames_per_batch=fpb,
            pyramid_levels=pyramid_levels,
            anchor_params=anchor_params)

    retinanet_losses = losses.RetinaNetLosses(
        sigma=3.0,
        alpha=0.25,
        gamma=2.0,
        iou_threshold=0.5,
        mask_size=(28,28))

    loss = {
        'regression': retinanet_losses.regress_loss,
        'classification': retinanet_losses.classification_loss,
        'masks': retinanet_losses.mask_loss,
        'final_detection': retinanet_losses.final_detection_loss
        }

    model.compile(loss=loss, optimizer=optimizer)

    iou_threshold = 0.5
    score_threshold = 0.01
    max_detections = 100

    model.fit_generator(
        train_data,
        steps_per_epoch=X_train.shape[0] * X_train.shape[1]// batch_size,
        epochs=n_epoch,
        validation_data=val_data,
        validation_steps=X_test.shape[0] * X_test.shape[1]// batch_size,
        callbacks=[
            callbacks.LearningRateScheduler(lr_sched),
            callbacks.ModelCheckpoint(
                os.path.join(model_dir, model_name + '.h5'),
                monitor='val_loss',
                verbose=1,
                save_best_only=True,
                save_weights_only=False),
            RedirectModel(
                Evaluate(val_data,
                         iou_threshold=iou_threshold,
                         score_threshold=score_threshold,
                         max_detections=max_detections,
                         frames_per_batch=fpb,
                         weighted_average=True),
                prediction_model)
        ])

    return None

# Train models

In [ ]:
# download_datasets()

DATA_DIR = '/data/training_data/tracking_benchmark_data'

backbones = ['resnet50']
fpbs = [5, 3, 1]
all_data = '3T3_HeLa_HEK_RAW_cropped.npz'
datasets = [all_data]
temporal_modes = ['conv', 'gru', 'lstm', None]
shape_mask = False

n_epoch = 4
seed = 808


for dataset in datasets:
    num_classes=1
    test_size = 0.1 # % of data saved as test
    test_seed = 10

    filename = os.path.join(DATA_DIR, dataset)
    train_dict, test_dict = get_data(filename, seed=seed, test_size=test_size)
    print(' -\nX.shape: {}\ny.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))
    X_train, y_train = train_dict['X'], train_dict['y']
    X_test, y_test = test_dict['X'], test_dict['y']
    y_train_reshaped = y_train.reshape((-1,  X_train.shape[2], X_train.shape[3], X_train.shape[4]))
    print("y_train_reshaped shape:", y_train_reshaped.shape)
    optimal_params = get_anchor_parameters(y_train.reshape((-1,  X_train.shape[2], X_train.shape[3], X_train.shape[4])))
    backbone_levels, pyramid_levels, anchor_params = optimal_params
    norm_method='whole_image'
    print("optimal_params: ", optimal_params)

    for backbone in backbones:
        if backbone == 'featurenet':
            use_imagenet=False
        else:
            use_imagenet=True

        for fpb in fpbs:
            if fpb == 1:
                train_dict = {'X':X_train.reshape((-1,X_train.shape[2], X_train.shape[3], X_train.shape[4])), 
                              'y': y_train.reshape((-1,y_train.shape[2], y_train.shape[3], y_train.shape[4]))}
                test_dict = {'X':X_test.reshape((-1, X_test.shape[2], X_test.shape[3], X_test.shape[4])), 
                            'y': y_test.reshape((-1, y_test.shape[2], y_test.shape[3], y_test.shape[4]))}
            else:
                train_dict = {'X':X_train, 'y':y_train}
                test_dict = {'X':X_test, 'y':y_test}
            print(' -\nX.shape: {}\ny.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))


            for temporal_mode in temporal_modes:
                model = model_zoo.RetinaMask(backbone=backbone,
                                        use_imagenet=use_imagenet,
                                        panoptic=False,
                                        frames_per_batch=fpb,
                                        temporal_mode=temporal_mode,
                                        num_classes=num_classes,
                                        input_shape=X_train.shape[2:],
                                        anchor_params=anchor_params,
                                        class_specific_filter=False,
                                        backbone_levels=backbone_levels,
                                        pyramid_levels=pyramid_levels,
                                        norm_method=norm_method)
                prediction_model = model

                model_dir = '/data/models'
                model_name = backbone + '_' + 'fpb' + str(fpb) + '_' + str(temporal_mode) + '_' + dataset

                # Train model
                print("Training model: ", model_name)
                trained_model = train_model(model,
                            model_dir=model_dir,
                            model_name=model_name,
                            train_dict=train_dict,
                            test_dict=test_dict,
                            fpb=fpb,
                            backbone_levels=backbone_levels,
                            pyramid_levels=pyramid_levels,
                            anchor_params=anchor_params,
                            n_epoch=n_epoch,
                          )


 -
X.shape: (722, 30, 135, 160, 1)
y.shape: (722, 30, 135, 160, 1)
y_train_reshaped shape: (21660, 135, 160, 1)


W0111 03:11:13.316836 140242619287360 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


optimal_params:  (['C1', 'C2', 'C3'], ['P1', 'P2', 'P3'], <deepcell.utils.retinanet_anchor_utils.AnchorParameters object at 0x7f8ca4023828>)
 -
X.shape: (722, 30, 135, 160, 1)
y.shape: (722, 30, 135, 160, 1)
94773248/94765736 [==============================] - 4s 0us/step


W0111 03:11:59.796795 140242619287360 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0111 03:12:14.272167 140242619287360 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:255: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0111 03:12:15.545627 140242619287360 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:255: calling crop_and_resize_v1 (from tensorflow.python.ops.image_ops_impl) with box_i

Training model:  resnet50_fpb5_conv_3T3_HeLa_HEK_RAW_cropped.npz


W0111 03:12:26.018308 140242619287360 retinanet.py:619] Removing 77 of 722 images with fewer than 3 objects.
W0111 03:12:29.030066 140242619287360 retinanet.py:619] Removing 7 of 81 images with fewer than 3 objects.
W0111 03:12:29.264892 140242619287360 training_utils.py:1101] Output filtered_detections missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to filtered_detections.
W0111 03:12:29.265924 140242619287360 training_utils.py:1101] Output filtered_detections_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to filtered_detections_1.
W0111 03:12:29.266691 140242619287360 training_utils.py:1101] Output filtered_detections_2 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to filtered_detections_2.
W0111 03:12:29.267596 1402426

Epoch 1/4
 8251/21660 [==========>...................] - ETA: 3:32:03 - loss: 1.8646 - regression_loss: 1.0828 - classification_loss: 0.2006 - masks_loss: 0.3234 - final_detection_loss: 0.2577

# Benchmark

In [12]:
# Define data to load (raw images from trk test files)
RAW_BASE_DIR = '/data/training_data/tracking_benchmark_data/test'

raw_trks_3T3  = os.path.join(RAW_BASE_DIR, '3T3_NIH_test_BData.trks')
raw_trks_HEK  = os.path.join(RAW_BASE_DIR, 'HEK293_generic_test_BData.trks')
raw_trks_HeLa = os.path.join(RAW_BASE_DIR, 'HeLa_S3_test_BData.trks')
raw_trks_RAW  = os.path.join(RAW_BASE_DIR, 'RAW264_generic_test_BData.trks')

raw_trks_files = [raw_trks_HeLa]
# raw_trks_files = [raw_trks_HeLa, raw_trks_RAW, raw_trks_3T3, raw_trks_HEK]

model_dir = '/data/models'

DATA_DIR = '/data/training_data/tracking_benchmark_data'
dataset = '3T3_HeLa_HEK_RAW_cropped.npz'
filename = os.path.join(DATA_DIR, dataset)
test_size = 0.1 # % of data saved as test
seed = 808
train_dict, test_dict = get_data(filename, seed=seed, test_size=test_size)
X_train, y_train = train_dict['X'], train_dict['y']

In [13]:
from skimage.segmentation import relabel_sequential
from skimage.measure import label, regionprops
from skimage.transform import resize


def retinanet_to_label_image(retinanet_outputs,
                             shape0,
                             shape1,
                             include_final_detection_layer=False,
                             score_threshold=0.5,
                             multi_iou_threshold=0.25,
                             binarize_threshold=0.5,
                             small_objects_threshold=100):

    if include_final_detection_layer:
        boxes_batch = retinanet_outputs[-5][0]
        scores_batch = retinanet_outputs[-4][0]
        labels_batch = retinanet_outputs[-3][0]
        masks_batch = retinanet_outputs[-2][0]
        final_scores = retinanet_outputs[-1][0]
    else:
        boxes_batch = retinanet_outputs[-4][0]
        scores_batch = retinanet_outputs[-3][0]
        labels_batch = retinanet_outputs[-2][0]
        masks_batch = retinanet_outputs[-1][0]
        
    # Create empty label matrix
    label_images = np.zeros(
        (masks_batch.shape[0], shape0, shape1))

    # Iterate over batches
    for i in range(boxes_batch.shape[0]):
        boxes = boxes_batch[i]
        scores = scores_batch[i]
        labels = labels_batch[i]
        masks = masks_batch[i]
        
        # Get good detections
        selection = np.nonzero(scores > score_threshold)[0]
        boxes = boxes[selection]
        scores = scores[selection]
        labels = labels[selection]
        masks = masks[selection, ..., -1]

        # Compute overlap of masks with each other
        mask_image = np.zeros((masks.shape[0], shape0, shape1), dtype='float32')

        for j in range(masks.shape[0]): # masks.shape[0] = fpb
            mask = masks[j]
            box = boxes[j].astype(int)
            mask = resize(mask, (box[3] - box[1], box[2] - box[0]))
            mask = (mask > binarize_threshold).astype('float32')
            mask_image[j, box[1]:box[3], box[0]:box[2]] = mask
            
        all_masks = mask_image
        range_mask = np.arange(1, all_masks.shape[0] + 1)
        all_masks *= np.expand_dims(np.expand_dims(range_mask, axis=-1), axis=-1)

        label_image = np.sum(all_masks, axis=0).astype(int)

        # Remove small objects
        label_image = morphology.remove_small_objects(
            label_image, min_size=small_objects_threshold)

        # Relabel the label image
        label_image, _, _ = relabel_sequential(label_image)

        # Store in batched array
        label_images[i] = label_image

    return label_images

In [14]:
# Make predictions on test data
from deepcell.utils.tracking_utils import load_trks
from deepcell.utils.retinanet_anchor_utils import evaluate

from skimage.morphology import remove_small_objects
import pandas as pd
from deepcell import metrics
from skimage.measure import label
from skimage import morphology
from skimage.morphology import watershed
from skimage.feature import peak_local_max


iou_threshold = 0.5
score_threshold = 0.01
max_detections = 100
num_classes=1

backbones = ['resnet50', 'mobilenetv2', 'featurenet']
fpbs = [5, 3]

temporal_modes = [None, 'lstm', 'conv', 'gru']
shape_mask = False
training_optimal_params = get_anchor_parameters(y_train.reshape((-1,  X_train.shape[2], X_train.shape[3], X_train.shape[4])))

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(raw_trks_files):
    print("dataset: ", dataset)
    # Load the trk file       
    trks = load_trks(dataset)
    lineages, raw, tracked = trks['lineages'], trks['X'], trks['y']
    norm_method='whole_image'
    backbone_levels, pyramid_levels, anchor_params = training_optimal_params
    datagen_val = RetinaMovieDataGenerator()
    
    for backbone in backbones:
        if backbone == 'featurenet':
                use_imagenet=False
        else:
            use_imagenet=True
                
        for fpb in fpbs:
            print("frames per batch: ", fpb)
            for temporal_mode in temporal_modes:
                prediction_model = model_zoo.RetinaMask(backbone=backbone,
                                        use_imagenet=use_imagenet,
                                        panoptic=False,
                                        frames_per_batch=fpb,
                                        temporal_mode=temporal_mode,
                                        num_classes=num_classes,
                                        input_shape=trks['X'].shape[2:],
                                        anchor_params=anchor_params,
                                        class_specific_filter=False,
                                        backbone_levels=backbone_levels,
                                        pyramid_levels=pyramid_levels,
                                        norm_method=norm_method)
                # print(prediction_model.summary())

                model_dir = '/data/models/'
                model_name = backbone + '_' + 'fpb' + str(fpb) + \
                            '_' + str(temporal_mode) + '_' + '3T3_HeLa_HEK_RAW_cropped.npz'
                print(backbone + '_' + 'fpb' + str(fpb) + '_' + str(temporal_mode))
                prediction_model.load_weights(os.path.join(model_dir, model_name + '.h5'))

                Model_DF = pd.DataFrame(columns=['total_instances', 'mAP'])
                
                Lstats_allmovies = []

                # Go through each batch (movie) in each dataset
                for batch_num, movie in enumerate(trks['X']):
                    Lstats = []
                    print("batch_num: ", batch_num)

                    # Predict on the raw data
                    X_test_temp = np.expand_dims(movie, axis=0)
                    y_test_temp = np.expand_dims(trks['y'][batch_num], axis=0)
                    y_pred = np.zeros(y_test_temp.shape)
                    
                    for index in range(X_test_temp.shape[0]):
                        for frame in range(0, X_test_temp.shape[1], fpb):
                            image = X_test_temp[index:index + 1, frame:frame + fpb]
                            gt_mask = y_test_temp[index:index + 1, frame:frame + fpb]
                            outputs = prediction_model.predict(image)
                            try:
                                label_images = retinanet_to_label_image(outputs, X_test_temp.shape[2], 
                                                   X_test_temp.shape[3], include_final_detection_layer=True)
                            
                                for i in range(len(label_images)): # len(label_images) = fpb
                                    GT_image = y_test_temp[index, frame+i, :, :, 0]
                                    GT_image = np.expand_dims(GT_image, axis=0)
                                    pred = label_images[i]
                                    pred = np.expand_dims(pred, axis=0)
                                    m = metrics.Metrics(model_name = model_name)
                                    m.calc_object_stats(GT_image, pred)
                                    Lstats.append(m.stats)

                                y_pred[index, frame:frame + fpb,...,0] = np.array(label_images)
                                
                            except:
                                continue
                            
                    Lstats_allmovies.append(Lstats)
                df = pd.concat([pd.concat(Lstats_allmovies[i]) for i in range(len(Lstats_allmovies)) \
                                if len(Lstats_allmovies[i]) >= 1])
                df.to_csv(os.getcwd() + '/' + dataset[len(RAW_BASE_DIR):len(RAW_BASE_DIR) +7] +\
                          '_' + backbone + '_' + \
                          'fpb' + str(fpb) + '_' + str(temporal_mode) + '.csv', index=False)
                print('\n')
                print(backbone + '_' + 'fpb' + str(fpb) + '_' + str(temporal_mode))
                print('\n')
                # Total number of correct detections and incorrect detections
                correct_det = int(df['correct_detections'].sum())
                incorrect_det = int(df['n_pred'].sum() - df['correct_detections'].sum())

                print('Correct Detections: ', correct_det)
                print('Incorrect Detections: ', incorrect_det)

                # Total number of splits, merges, and catastrophes
                splits = df['split'].sum()
                merges = df['merge'].sum()
                catastrophes = df['catastrophe'].sum()

                print('Splits: ', splits)
                print('Merges: ', merges)
                print('Catastrophes: ', catastrophes)

                # Average Recall, Precision, and Jaccard Index
                recall = 100 * df['correct_detections'].sum() / df['n_true'].sum()
                precision = 100 * df['correct_detections'].sum() / df['n_pred'].sum()
                jaccard = df['jaccard'].mean()

                print('Recall: ', recall)
                print('Precision: ', precision)
                print('Average Jaccard Index: ', jaccard)
                print('F1: ', 2 * recall * precision/ (precision + recall))
            

dataset:  /data/training_data/tracking_benchmark_data/test/HeLa_S3_test_BData.trks
frames per batch:  5
resnet50_fpb5_None
batch_num:  0

____________Object-based statistics____________

Number of true cells:		 10
Number of predicted cells:	 8

Correct detections:  8	Recall: 80%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 2	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.81411492479753178042045647089253179728984832763671875 


____________Object-based statistics____________

Number of true cells:		 10
Number of predicted cells:	 9

Correct detections:  7	Recall: 70%
Incorrect detections: 2	Precision: 77.77777777777777146184234879910945892333984375%

Gained detections: 0	Perc Error: 0%
Mi


____________Object-based statistics____________

Number of true cells:		 9
Number of predicted cells:	 8

Correct detections:  8	Recall: 88.888888888888885730921174399554729461669921875%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 1	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.8272342064714945930603562374017201364040374755859375 


____________Object-based statistics____________

Number of true cells:		 9
Number of predicted cells:	 8

Correct detections:  8	Recall: 88.888888888888885730921174399554729461669921875%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 1	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Cata


____________Object-based statistics____________

Number of true cells:		 8
Number of predicted cells:	 8

Correct detections:  8	Recall: 100%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: NaN%
Missed detections: 0	Perc Error: NaN%
Merges: 0		Perc Error: NaN%
Splits: 0		Perc Error: NaN%
Catastrophes: 0		Perc Error: NaN%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.79807503437438620519372989292605780065059661865234375 


____________Object-based statistics____________

Number of true cells:		 8
Number of predicted cells:	 8

Correct detections:  8	Recall: 100%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: NaN%
Missed detections: 0	Perc Error: NaN%
Merges: 0		Perc Error: NaN%
Splits: 0		Perc Error: NaN%
Catastrophes: 0		Perc Error: NaN%

Gained detections from splits: 0
Missed dete


____________Object-based statistics____________

Number of true cells:		 12
Number of predicted cells:	 12

Correct detections:  12	Recall: 100%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: NaN%
Missed detections: 0	Perc Error: NaN%
Merges: 0		Perc Error: NaN%
Splits: 0		Perc Error: NaN%
Catastrophes: 0		Perc Error: NaN%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.812864864864864866689231348573230206966400146484375 


____________Object-based statistics____________

Number of true cells:		 12
Number of predicted cells:	 12

Correct detections:  12	Recall: 100%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: NaN%
Missed detections: 0	Perc Error: NaN%
Merges: 0		Perc Error: NaN%
Splits: 0		Perc Error: NaN%
Catastrophes: 0		Perc Error: NaN%

Gained detections from splits: 0
Missed 


____________Object-based statistics____________

Number of true cells:		 12
Number of predicted cells:	 12

Correct detections:  11	Recall: 91.6666666666666714036182384006679058074951171875%
Incorrect detections: 1	Precision: 91.6666666666666714036182384006679058074951171875%

Gained detections: 1	Perc Error: 50%
Missed detections: 1	Perc Error: 50%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.7906378600823045221801521620363928377628326416015625 


____________Object-based statistics____________

Number of true cells:		 12
Number of predicted cells:	 12

Correct detections:  12	Recall: 100%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: NaN%
Missed detections: 0	Perc Error: NaN%
Merges: 0		Perc Error: NaN%
Splits: 0		Perc Er

...


____________Object-based statistics____________

Number of true cells:		 6
Number of predicted cells:	 6

Correct detections:  6	Recall: 100%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: NaN%
Missed detections: 0	Perc Error: NaN%
Merges: 0		Perc Error: NaN%
Splits: 0		Perc Error: NaN%
Catastrophes: 0		Perc Error: NaN%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.863686902379364074278100815718062222003936767578125 


____________Object-based statistics____________

Number of true cells:		 6
Number of predicted cells:	 6

Correct detections:  6	Recall: 100%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: NaN%
Missed detections: 0	Perc Error: NaN%
Merges: 0		Perc Error: NaN%
Splits: 0		Perc Error: NaN%
Catastrophes: 0		Perc Error: NaN%

Gained detections from splits: 0
Missed detect


____________Object-based statistics____________

Number of true cells:		 2
Number of predicted cells:	 1

Correct detections:  1	Recall: 50%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 1	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.738491674828599453661581719643436372280120849609375 


____________Object-based statistics____________

Number of true cells:		 2
Number of predicted cells:	 1

Correct detections:  1	Recall: 50%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 1	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges: 


____________Object-based statistics____________

Number of true cells:		 2
Number of predicted cells:	 1

Correct detections:  1	Recall: 50%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 1	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.801876955161626714385647574090398848056793212890625 


____________Object-based statistics____________

Number of true cells:		 2
Number of predicted cells:	 1

Correct detections:  1	Recall: 50%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 1	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges: 


____________Object-based statistics____________

Number of true cells:		 2
Number of predicted cells:	 1

Correct detections:  1	Recall: 50%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 1	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges: 0
True detections involved in catastrophes: 0
Predicted detections involved in catastrophes: 0 

Average Pixel IOU (Jaccard Index): 0.85508849557522126350050939436187036335468292236328125 


____________Object-based statistics____________

Number of true cells:		 2
Number of predicted cells:	 1

Correct detections:  1	Recall: 50%
Incorrect detections: 0	Precision: 100%

Gained detections: 0	Perc Error: 0%
Missed detections: 1	Perc Error: 100%
Merges: 0		Perc Error: 0%
Splits: 0		Perc Error: 0%
Catastrophes: 0		Perc Error: 0%

Gained detections from splits: 0
Missed detections from merges

KeyboardInterrupt: 

In [ ]:
# HEK

In [9]:
temp = pd.read_csv('3T3_NI_mobilenetv2_fpb3_conv.csv')
df = temp
correct_det = int(df['correct_detections'].sum())
incorrect_det = int(df['n_pred'].sum() - df['correct_detections'].sum())

print('Correct Detections: ', correct_det)
print('Incorrect Detections: ', incorrect_det)

# Total number of splits, merges, and catastrophes
splits = df['split'].sum()
merges = df['merge'].sum()
catastrophes = df['catastrophe'].sum()

print('Splits: ', splits)
print('Merges: ', merges)
print('Catastrophes: ', catastrophes)

# Average Recall, Precision, and Jaccard Index
recall = 100 * df['correct_detections'].sum() / df['n_true'].sum()
precision = 100 * df['correct_detections'].sum() / df['n_pred'].sum()
jaccard = df['jaccard'].mean()

print('Recall: ', recall)
print('Precision: ', precision)
print('Average Jaccard Index: ', jaccard)
print('F1: ', 2 * recall * precision/ (precision + recall))

Correct Detections:  5762
Incorrect Detections:  669
Splits:  102
Merges:  7
Catastrophes:  5
Recall:  82.09146602080068
Precision:  89.59726325610325
Average Jaccard Index:  0.7499341034879876
F1:  85.68029739776952
